In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [27]:
df=pd.read_csv(r"D:\Swiggy_Time_prediction\Data\encoded.csv")
df.head()

,Unnamed: 0,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,traffic,vehicle_condition,multiple_deliveries,...,weather_sandstorms,weather_stormy,weather_sunny,weather_windy,type_of_order_drinks,type_of_order_meal,type_of_order_snack,type_of_vehicle_electric_scooter,type_of_vehicle_motorcycle,type_of_vehicle_scooter
0,0,37.0,0.741937,22.745049,4.342408,22.765049,4.342668,2.0,2,0.0,...,False,False,True,False,False,False,True,False,True,False
1,1,34.0,0.916291,12.913041,4.365430,13.043041,4.367081,3.0,2,1.0,...,False,True,False,False,False,False,True,False,False,True
2,2,23.0,0.955511,12.914264,4.365369,12.924264,4.365496,0.0,0,1.0,...,True,False,False,False,True,False,False,False,True,False
3,3,38.0,0.832909,11.003669,4.356407,11.053669,4.357048,1.0,0,1.0,...,False,False,True,False,False,False,False,False,True,False
4,4,32.0,0.875469,12.972793,4.397531,13.012793,4.398023,2.0,1,1.0,...,False,False,False,False,False,False,True,False,False,True


In [28]:
df.drop(columns=["Unnamed: 0"],inplace=True)

In [29]:
selected_features = [
    "traffic",
    "distance",
    "ratings",
    "age",
    "vehicle_condition",
    "multiple_deliveries",
    "order_time_hour",
    "order_day",
    "city_type",
    "festival",
    "pickup_time_minutes",
    "weather_sunny",
    "weather_stormy",
    "weather_windy",
    "weather_sandstorms",
    "weather_fog"
]

In [30]:
from sklearn.model_selection import train_test_split

X = df[selected_features]
y = df["time_taken"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [32]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

y_pred_lr = lr.predict(X_test_scaled)


In [33]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(
    n_neighbors=7,
    weights="distance"
)

knn.fit(X_train_scaled, y_train)

y_pred_knn = knn.predict(X_test_scaled)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

X = df[selected_features]
y = df["time_taken"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

rf = RandomForestRegressor(
    min_samples_split=10,
    min_samples_leaf=1,
    max_features='log2',
    n_estimators=300,
    random_state=42,
    n_jobs=1
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)


In [36]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


def evaluate(y_true, y_pred):
    return {
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "R2": r2_score(y_true, y_pred)
    }

results = pd.DataFrame.from_dict({
    "Linear Regression": evaluate(y_test, y_pred_lr),
    "KNN": evaluate(y_test, y_pred_knn),
    "Random Forest": evaluate(y_test, y_pred_rf)
}, orient="index")

results

,MAE,RMSE,R2
Linear Regression,5.012256,6.314297,0.546727
KNN,3.927935,5.084989,0.706039
Random Forest,3.278211,4.159829,0.803274
